In [2]:
import librosa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import io
import gc
import os
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
import tensorflow as tf
import seaborn as sns
from scipy.signal import butter, lfilter
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Flatten, MaxPooling2D, Conv2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.activations import relu, tanh, linear
from tensorflow.keras.utils import Progbar
from joblib import Parallel, delayed #Paralleize calculation
import psycopg2
from sqlalchemy import create_engine, Column, Integer, ARRAY, MetaData, Table, Text
from sqlalchemy.dialects.postgresql import ARRAY as PG_ARRAY
from psycopg2.extensions import register_adapter, AsIs
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from tensorflow.keras.backend import clear_session

# Set random seed for reproducability
tf.keras.utils.set_random_seed(42)
# check if tensorflow uses GPU
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
# Custom adapter function for postgre
def adapt_numpy_ndarray(numpy_array):
    return AsIs(list(numpy_array))
# Register the postgre-adapter
register_adapter(np.ndarray, adapt_numpy_ndarray)

# Database connection parameters and alchemy engine
dbname = 'bathunting'
user = 'python'
password = 'python_password'
host = 'localhost'
port = '5432' 
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}')

# Functions to get datatf.compat.v1.reset_default_graph()  # Reset the TensorFlow graph

def get_target_data(target, limit=0, no_target=False):
    lmt = "" if limit<=0 else f"LIMIT {limit}"
    #query = ""
    if no_target:
        query = f"SELECT new_arr FROM batcall where 10 < ANY(new_arr) and target = {target} {lmt}"
    else:
        query = f"SELECT target, new_arr FROM batcall where 10 < ANY(new_arr) and target = {target} {lmt}"
    df = pd.read_sql_query(query, engine)
    if no_target:
        df = pd.DataFrame(df['new_arr'].tolist())
    return df

# messy, i know
def get_targets_to_data(limit=0):
    lmt = "" if limit<=0 else f"LIMIT {limit}"
    query = f"SELECT target FROM batcall where 10 < ANY(new_arr) {lmt}"
    df = pd.read_sql_query(query, engine)
    return df

def get_all_data(targets=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18], limit=0, no_target=False):
    all_df = Parallel(n_jobs=-3, prefer="threads")(delayed(get_target_data)(target, limit, no_target) for target in targets)
    return all_df

def get_data(targets=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18], limit=0, no_target=False):
    all_df = Parallel(n_jobs=-3, prefer="threads")(delayed(get_target_data)(target, limit, no_target) for target in targets)
    df = pd.concat(all_df)
    return df

# try to visualize only the maximum values per species
def spectrogram_range(target):
    df = get_target_data(target, limit=0, no_target=True)
    max_vals = df.max()
    min_vals = df.min()
    
    abs_pos = max_vals.abs()
    abs_neg = min_vals.abs()
    mask = (abs_pos > abs_neg).astype(bool)
    return min_vals.where(mask, max_vals), max_vals.where(mask, min_vals)

def get_targets():
    #conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host)
    #cursor = conn.cursor()
    query = f"SELECT target, bat FROM batcall group by target, bat order by target"
    df = pd.read_sql_query(query, engine)
    #conn.close()
    return df

def get_shape(nested_list):
    try:
        # Initialize shape list
        shape = []

        # Iterate to calculate the shape
        while isinstance(nested_list, list) or isinstance(nested_list, np.ndarray):
            shape.append(len(nested_list))
            nested_list = nested_list[0]

        return tuple(shape)
    except (TypeError, IndexError) as e:
        # In case the nested lists are not uniformly sized
        return f"Irregular shape - nested lists are not of equal size. \n ERROR: {e}"
    
# Get data to work with
def get_features_and_targets(limit=100, scaler=StandardScaler(), categorical=True):
    data = get_data(limit=limit)

    df = pd.DataFrame(data["new_arr"].tolist())
    if scaler != None:
        df = scaler.fit_transform(df)

    labels = pd.DataFrame(data["target"])
    if categorical:
        labels = to_categorical(labels, num_classes=19)
    return df, labels

def vogl_conversion(df):
    data_reshaped = []
    for _,data in df.iterrows():
        # Normalize
        data -= np.mean(data)
        data /= np.std(data)
        # Realy no idea just assuming prof did it right
        # Calculate spectrogram with FFT
        stft = np.abs(librosa.stft(np.array(data), n_fft=512, hop_length=32))
        stft = 10 * np.log10(stft)
        stft = np.nan_to_num(stft)
        # Scale between [0,1] and reduce shape if needed
        stft = (stft - np.min(stft)) / (np.max(stft) - np.min(stft))
        stft = np.reshape(stft, (257, 138, 1))
        stft = stft[:256, -128: , :]
        data_reshaped.append(stft)
    return np.array(data_reshaped)

def confusion_matrix(model, X_test, y_test):
    # Confusion Matrix
    predictions = model.predict(X_test)
    predicted_labels = np.argmax(predictions, axis=1)
    true_labels = np.argmax(y_test, axis=1)  # y_test are the true labels (one-hot encoded)
    conf_matrix = confusion_matrix(true_labels, predicted_labels)

    # Plot the confusion matrix using Seaborn
    sns.heatmap(conf_matrix, annot=True, fmt='g')
    plt.xlabel('Predicted labels')
    plt.ylabel('True labels')
    plt.show()

def visualize_history(history, title_appendix=''):
    # Plot training & validation accuracy values
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title(f'Model accuracy {title_appendix}')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.savefig(f'History/accuracy_{title_appendix}.png')
    plt.close()
    
    # Plot training & validation loss values
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(f'Model loss {title_appendix}')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.savefig(f'History/loss_{title_appendix}.png')
    plt.close()

This should be all about Convolutional Neuronal Networks and lowering the loss

In [4]:
df, labels = get_features_and_targets(limit=500, scaler=None)
df_reshaped = vogl_conversion(df)
X_train, X_test, y_train, y_test = train_test_split(df_reshaped, labels, test_size=0.2, random_state=42)
X_train = X_train[:-3]
y_train = y_train[:-3]
print(get_shape(X_train))
print(get_shape(y_train))
# Determine the number of input features
input_dim = get_shape(X_train)

/tmp/ipykernel_2495/927267477.py:101: RuntimeWarning: divide by zero encountered in log10
  stft = 10 * np.log10(stft)


(7250, 256, 128, 1)
(7250, 19)


In [13]:
# Density 50 to 5000 -> does not realy matter after 100 and not matter anymore after 500
for i in range(50, 5001, 50):
    clear_session() # clear keras session
    tf.compat.v1.reset_default_graph()  # Reset the TensorFlow graph
    #tf.config.gpu.set_per_process_memory_fraction(0.5)
    # Build the model
    model = Sequential()
    model.add(Conv2D(16, (3,3), activation='relu', padding='same', input_shape=(256,128,1)))
    model.add(MaxPooling2D(2, 2))
    model.add(Flatten())
    model.add(Dense(i, activation='relu'))
    model.add(Dense(19, activation='softmax'))

    # Compile the model
    model.compile(loss='categorical_crossentropy',
                  optimizer=RMSprop(learning_rate=0.001),
                  metrics=['accuracy'])

    # Fit the model
    history = model.fit(X_train, y_train, epochs=50, batch_size=10, verbose=0, validation_split=0.2)

    # Evaluate the model
    loss, accuracy = model.evaluate(X_test, y_test)
    print('Density: %.i' % (i))
    print(f"Final Training Loss: {history.history['loss'][-1]}")
    print(f"Final Training Accuracy: {history.history['accuracy'][-1]}")
    print(f"Final Validation Loss: {history.history['val_loss'][-1]}")
    print(f"Final Validation Accuracy: {history.history['val_accuracy'][-1]}")
    visualize_history(history, f'dens={i}_finacc={history.history["val_accuracy"][-1]:.2f}')
    
    # Free Memory for following models
    del model
    gc.collect()
    tf.compat.v1.reset_default_graph()  # Reset the TensorFlow graph

57/57 [==============================] - 0s 6ms/step - loss: 2.9404 - accuracy: 0.0485
Density: 50
Final Training Loss: 2.9372239112854004
Final Training Accuracy: 0.05155172571539879
Final Validation Loss: 2.9394729137420654
Final Validation Accuracy: 0.05517241358757019
Test Accuracy: 4.85 %
Test Loss: 2.94
57/57 [==============================] - 0s 5ms/step - loss: 3.0835 - accuracy: 0.6985
Density: 100
Final Training Loss: 1.479933757764229e-06
Final Training Accuracy: 1.0
Final Validation Loss: 2.7108476161956787
Final Validation Accuracy: 0.7103448510169983
Test Accuracy: 69.85 %
Test Loss: 3.08
57/57 [==============================] - 0s 5ms/step - loss: 1.7402 - accuracy: 0.7800
Density: 150
Final Training Loss: 6.372534357979021e-07
Final Training Accuracy: 1.0
Final Validation Loss: 1.7338581085205078
Final Validation Accuracy: 0.7731034755706787
Test Accuracy: 78.00 %
Test Loss: 1.74
57/57 [==============================] - 1s 7ms/step - loss: 1.7007 - accuracy: 0.7778
Dens

KeyboardInterrupt: 

In [5]:
# increase convolutional filters to 32
# decrease epochs to 25
# Density 20 to 2000
# -> accuracy is fine at density about 200, loss is low and neighter accuracy nore loss change significantly afterwards. 
# Also overfitting seems to happen after about 4 or 5 epochs
title_appendix = 'e25_1x_conv32'
for i in range(20, 1001, 20):
    clear_session() # clear keras session
    tf.compat.v1.reset_default_graph()  # Reset the TensorFlow graph
    #tf.config.gpu.set_per_process_memory_fraction(0.5)
    # Build the model
    model = Sequential()
    model.add(Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(256,128,1)))
    model.add(MaxPooling2D(2, 2))
    model.add(Flatten())
    model.add(Dense(i, activation='relu'))
    model.add(Dense(19, activation='softmax'))

    # Compile the model
    model.compile(loss='categorical_crossentropy',
                  optimizer=RMSprop(learning_rate=0.001),
                  metrics=['accuracy'])

    # Fit the model
    history = model.fit(X_train, y_train, epochs=25, batch_size=10, verbose=0, validation_split=0.2)

    # Evaluate the model
    loss, accuracy = model.evaluate(X_test, y_test)
    print('Density: %.i' % (i))
    print(f"Final Training Loss: {history.history['loss'][-1]}")
    print(f"Final Training Accuracy: {history.history['accuracy'][-1]}")
    print(f"Final Validation Loss: {history.history['val_loss'][-1]}")
    print(f"Final Validation Accuracy: {history.history['val_accuracy'][-1]}")
    visualize_history(history, f'{title_appendix}_dens={i}_finacc={history.history["val_accuracy"][-1]:.2f}')
    
    # Free Memory for following models
    del model
    gc.collect()
    tf.compat.v1.reset_default_graph()  # Reset the TensorFlow graph

57/57 [==============================] - 1s 7ms/step - loss: 2.9405 - accuracy: 0.0496
Density: 20
Final Training Loss: 2.937248706817627
Final Training Accuracy: 0.05672413855791092
Final Validation Loss: 2.939340353012085
Final Validation Accuracy: 0.05241379141807556
Test Accuracy: 4.96 %
Test Loss: 2.94
57/57 [==============================] - 0s 6ms/step - loss: 2.9405 - accuracy: 0.0496
Density: 40
Final Training Loss: 2.937248706817627
Final Training Accuracy: 0.05672413855791092
Final Validation Loss: 2.939340353012085
Final Validation Accuracy: 0.05241379141807556
Test Accuracy: 4.96 %
Test Loss: 2.94
57/57 [==============================] - 0s 6ms/step - loss: 2.9405 - accuracy: 0.0496
Density: 60
Final Training Loss: 2.937248706817627
Final Training Accuracy: 0.05672413855791092
Final Validation Loss: 2.939340591430664
Final Validation Accuracy: 0.05241379141807556
Test Accuracy: 4.96 %
Test Loss: 2.94
57/57 [==============================] - 0s 7ms/step - loss: 2.9405 - acc

KeyboardInterrupt: 

In [6]:
# increase convolutional filters to 64
# decrease epochs to 20
# Density 150 to 500
title_appendix = 'e20_1x_conv64'
for i in range(150, 501, 10):
    clear_session() # clear keras session
    tf.compat.v1.reset_default_graph()  # Reset the TensorFlow graph
    #tf.config.gpu.set_per_process_memory_fraction(0.5)
    # Build the model
    model = Sequential()
    model.add(Conv2D(64, (3,3), activation='relu', paddine20_1x_conv64g='same', input_shape=(256,128,1)))
    model.add(MaxPooling2D(2, 2))
    model.add(Flatten())
    model.add(Dense(i, activation='relu'))
    model.add(Dense(19, activation='softmax'))

    # Compile the model
    model.compile(loss='categorical_crossentropy',
                  optimizer=RMSprop(learning_rate=0.001),
                  metrics=['accuracy'])

    # Fit the model
    history = model.fit(X_train, y_train, epochs=20, batch_size=10, verbose=0, validation_split=0.2)

    # Evaluate the model
    loss, accuracy = model.evaluate(X_test, y_test)
    print('Density: %.i' % (i))
    print(f"Final Training Loss: {history.history['loss'][-1]}")
    print(f"Final Training Accuracy: {history.history['accuracy'][-1]}")
    print(f"Final Validation Loss: {history.history['val_loss'][-1]}")
    print(f"Final Validation Accuracy: {history.history['val_accuracy'][-1]}")
    visualize_history(history, f'{title_appendix}_dens={i}_finacc={history.history["val_accuracy"][-1]:.2f}')
    
    # Free Memory for following models
    del model
    gc.collect()
    tf.compat.v1.reset_default_graph()  # Reset the TensorFlow graph

57/57 [==============================] - 1s 10ms/step - loss: 1.8074 - accuracy: 0.7541
Density: 150
Final Training Loss: 0.017362574115395546
Final Training Accuracy: 0.9960345029830933
Final Validation Loss: 1.6691466569900513
Final Validation Accuracy: 0.7558620572090149
Test Accuracy: 75.41 %
Test Loss: 1.81
57/57 [==============================] - 1s 10ms/step - loss: 1.8635 - accuracy: 0.7266
Density: 160
Final Training Loss: 0.013060218654572964
Final Training Accuracy: 0.9987931251525879
Final Validation Loss: 1.7736579179763794
Final Validation Accuracy: 0.7434482574462891
Test Accuracy: 72.66 %
Test Loss: 1.86
57/57 [==============================] - 1s 17ms/step - loss: 1.6025 - accuracy: 0.7674
Density: 170
Final Training Loss: 0.02390725165605545
Final Training Accuracy: 0.9941379427909851
Final Validation Loss: 1.483515739440918
Final Validation Accuracy: 0.7655172348022461
Test Accuracy: 76.74 %
Test Loss: 1.60
57/57 [==============================] - 1s 16ms/step - loss

2024-01-10 19:20:23.514166: E tensorflow/compiler/xla/stream_executor/gpu/gpu_cudamallocasync_allocator.cc:306] gpu_async_0 cuMemAllocAsync failed to allocate 671088640 bytes: CUDA error: out of memory (CUDA_ERROR_OUT_OF_MEMORY)
 Reported by CUDA: Free memory/Total memory: 67043328/8510832640
2024-01-10 19:20:23.514196: E tensorflow/compiler/xla/stream_executor/gpu/gpu_cudamallocasync_allocator.cc:311] Stats: Limit:                      7340097536
InUse:                      4619572912
MaxInUse:                   6574647020
NumAllocs:                    25780554
MaxAllocSize:                760217600
Reserved:                            0
PeakReserved:                        0
LargestFreeBlock:                    0

2024-01-10 19:20:23.514211: E tensorflow/compiler/xla/stream_executor/gpu/gpu_cudamallocasync_allocator.cc:63] Histogram of current allocation: (allocation_size_in_bytes, nb_allocation_of_that_sizes), ...;
2024-01-10 19:20:23.514217: E tensorflow/compiler/xla/stream_executo

ResourceExhaustedError: {{function_node __wrapped__AddV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} failed to allocate memory [Op:AddV2] name: 

Die Anzahl der Filter pro layer hat besten falls einen geringen Einfluss auf die Modelgüte.

Wie siehts mit der Lernrate von RMS-Propagation aus?

In [5]:
# Addapt RMS-Propagation learning rate
# convolutional filters: 32; epochs: 50; density: 250
title_appendix = 'l1_d50_d250'
for i in [0.01, 0.005, 0.001, 0.0005, 0.0001, 0.00005]:
    clear_session() # clear keras session
    tf.compat.v1.reset_default_graph()  # Reset the TensorFlow graph
    #tf.config.gpu.set_per_process_memory_fraction(0.5)
    # Build the model
    model = Sequential()
    model.add(Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(256,128,1)))
    model.add(MaxPooling2D(2, 2))
    model.add(Flatten())
    model.add(Dense(250, activation='relu'))
    model.add(Dense(19, activation='softmax'))

    # Compile the model
    model.compile(loss='categorical_crossentropy',
                  optimizer=RMSprop(learning_rate=0.001),
                  metrics=['accuracy'])

    # Fit the model
    history = model.fit(X_train, y_train, epochs=50, batch_size=10, verbose=0, validation_split=0.2)

    # Evaluate the model
    loss, accuracy = model.evaluate(X_test, y_test)
    print('RMS propagation learning rate: %.f' % (i))
    print(f"Final Training Loss: {history.history['loss'][-1]}")
    print(f"Final Training Accuracy: {history.history['accuracy'][-1]}")
    print(f"Final Validation Loss: {history.history['val_loss'][-1]}")
    print(f"Final Validation Accuracy: {history.history['val_accuracy'][-1]}")
    visualize_history(history, f'{title_appendix}_rmsplr={i}_finacc={history.history["val_accuracy"][-1]:.2f}')
    
    # Free Memory for following models
    del model
    gc.collect()
    tf.compat.v1.reset_default_graph()  # Reset the TensorFlow graph

2024-01-11 15:32:42.028380: I tensorflow/core/common_runtime/gpu/gpu_process_state.cc:227] Using CUDA malloc Async allocator for GPU: 0
2024-01-11 15:32:42.028506: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6995 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080, pci bus id: 0000:01:00.0, compute capability: 6.1
2024-01-11 15:32:45.328644: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8907
2024-01-11 15:32:46.927769: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fe591c38190 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-11 15:32:46.927807: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1080, Compute Capability 6.1
2024-01-11 15:32:46.984109: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env 

57/57 [==============================] - 1s 12ms/step - loss: 1.7850 - accuracy: 0.7894
Density: 0
Final Training Loss: 4.2470969674468506e-07
Final Training Accuracy: 1.0
Final Validation Loss: 1.7683717012405396
Final Validation Accuracy: 0.7641379237174988
Test Accuracy: 78.94 %
Test Loss: 1.78
57/57 [==============================] - 1s 11ms/step - loss: 1.8260 - accuracy: 0.7822
Density: 0
Final Training Loss: 5.756143650614831e-07
Final Training Accuracy: 1.0
Final Validation Loss: 1.7531464099884033
Final Validation Accuracy: 0.7793103456497192
Test Accuracy: 78.22 %
Test Loss: 1.83
57/57 [==============================] - 1s 11ms/step - loss: 1.7639 - accuracy: 0.7800
Density: 0
Final Training Loss: 4.3112447656312725e-07
Final Training Accuracy: 1.0
Final Validation Loss: 1.697001338005066
Final Validation Accuracy: 0.7793103456497192
Test Accuracy: 78.00 %
Test Loss: 1.76
57/57 [==============================] - 1s 11ms/step - loss: 1.7753 - accuracy: 0.7712
Density: 0
Final 

In [ ]:
# Addapt RMS-Propagation learning rate
# Repeate with 100 epochs
# convolutional filters: 32; epochs: 100; density: 250
title_appendix = 'l1_e100_d250'
for i in [0.01, 0.005, 0.001, 0.0005, 0.0001, 0.00005]:
    clear_session() # clear keras session
    tf.compat.v1.reset_default_graph()  # Reset the TensorFlow graph
    #tf.config.gpu.set_per_process_memory_fraction(0.5)
    # Build the model
    model = Sequential()
    model.add(Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(256,128,1)))
    model.add(MaxPooling2D(2, 2))
    model.add(Flatten())
    model.add(Dense(250, activation='relu'))
    model.add(Dense(19, activation='softmax'))

    # Compile the model
    model.compile(loss='categorical_crossentropy',
                  optimizer=RMSprop(learning_rate=0.001),
                  metrics=['accuracy'])

    # Fit the model
    history = model.fit(X_train, y_train, epochs=100, batch_size=10, verbose=0, validation_split=0.2)

    # Evaluate the model
    loss, accuracy = model.evaluate(X_test, y_test)
    print('RMS propagation learning rate: %.5f' % (i))
    print(f"Final Training Loss: {history.history['loss'][-1]}")
    print(f"Final Training Accuracy: {history.history['accuracy'][-1]}")
    print(f"Final Validation Loss: {history.history['val_loss'][-1]}")
    print(f"Final Validation Accuracy: {history.history['val_accuracy'][-1]}")
    visualize_history(history, f'{title_appendix}_rmsplr={i}_finacc={history.history["val_accuracy"][-1]:.2f}')
    
    # Free Memory for following models
    del model
    gc.collect()
    tf.compat.v1.reset_default_graph()  # Reset the TensorFlow graph

In [7]:
# Adam Optimizer with variing starting learn rates
# convolutional filters: 32; epochs: 50; density: 250, rms_propagation learning rate:
title_appendix = 'l1_e50_d250'
for i in [0.01, 0.005, 0.001, 0.0005, 0.0001, 0.00005]:
    clear_session() # clear keras session
    tf.compat.v1.reset_default_graph()  # Reset the TensorFlow graph
    #tf.config.gpu.set_per_process_memory_fraction(0.5)
    # Build the model
    model = Sequential()
    model.add(Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(256,128,1)))
    model.add(MaxPooling2D(2, 2))
    model.add(Flatten())
    model.add(Dense(250, activation='relu'))
    model.add(Dense(19, activation='softmax'))
    
    # Compile the model
    optimizer = tf.keras.optimizers.Adam(learning_rate=i)
    model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])
    
    # Fit the model
    history = model.fit(X_train, y_train, epochs=50, batch_size=10, verbose=0, validation_split=0.2)
    
    # Evaluate the model
    loss, accuracy = model.evaluate(X_test, y_test)
    print('Adam starting learning rate: %.5f' % (i))
    print(f"Final Training Loss: {history.history['loss'][-1]}")
    print(f"Final Training Accuracy: {history.history['accuracy'][-1]}")
    print(f"Final Validation Loss: {history.history['val_loss'][-1]}")
    print(f"Final Validation Accuracy: {history.history['val_accuracy'][-1]}")
    visualize_history(history, f'{title_appendix}_adam={i}_finacc={history.history["val_accuracy"][-1]:.2f}')
    
    # Free Memory for following models
    del model
    gc.collect()
    tf.compat.v1.reset_default_graph()  # Reset the TensorFlow graph

57/57 [==============================] - 1s 11ms/step - loss: 2.9395 - accuracy: 0.0562
Adam starting learning rate: 0
Final Training Loss: 2.9416658878326416
Final Training Accuracy: 0.056551724672317505
Final Validation Loss: 2.9402124881744385
Final Validation Accuracy: 0.053793102502822876
Test Accuracy: 5.62 %
Test Loss: 2.94
57/57 [==============================] - 1s 11ms/step - loss: 11.1687 - accuracy: 0.3611
Adam starting learning rate: 0
Final Training Loss: 0.1445300281047821
Final Training Accuracy: 0.9613792896270752
Final Validation Loss: 10.330340385437012
Final Validation Accuracy: 0.4000000059604645
Test Accuracy: 36.11 %
Test Loss: 11.17
57/57 [==============================] - 1s 11ms/step - loss: 1.9287 - accuracy: 0.6521
Adam starting learning rate: 0
Final Training Loss: 0.014938338659703732
Final Training Accuracy: 0.9967241287231445
Final Validation Loss: 1.7592754364013672
Final Validation Accuracy: 0.6731034517288208
Test Accuracy: 65.21 %
Test Loss: 1.93
57/

RMS-Propagation glänzt durch eine konstant gute Genauigkeit bei konstant relativ kleinem Loss wärend Adam in Sachen Genauigkeit weit zurück liegt und auch einen größeren loss vorweist. Da die Ergebnisse für verschiedene Start-Lernraten beim Adam-Optimizer stark abweichen muss das Model evtl. noch angepasst werden. 

Wie sieht's bei 100 Epochen und 500 Neuronen aus? 
<!-- Der Fairness halber wird das Model auch mit dem RMS-Propagation-Optimizer mit diesen Parametern überprüft. -->

In [8]:
# Adam Optimizer with variing starting learn rates
# convolutional filters: 32; epochs: 50; density: 250, rms_propagation learning rate:
title_appendix = 'l1_e100_d500'
for i in [0.01, 0.005, 0.001, 0.0005, 0.0001, 0.00005]:
    clear_session() # clear keras session
    tf.compat.v1.reset_default_graph()  # Reset the TensorFlow graph
    #tf.config.gpu.set_per_process_memory_fraction(0.5)
    # Build the model
    model = Sequential()
    model.add(Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(256,128,1)))
    model.add(MaxPooling2D(2, 2))
    model.add(Flatten())
    model.add(Dense(500, activation='relu'))
    model.add(Dense(19, activation='softmax'))
    
    # Compile the model
    optimizer = tf.keras.optimizers.Adam(learning_rate=i)
    model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])
    
    # Fit the model
    history = model.fit(X_train, y_train, epochs=100, batch_size=10, verbose=0, validation_split=0.2)
    
    # Evaluate the model
    loss, accuracy = model.evaluate(X_test, y_test)
    print('Adam starting learning rate: %.5f' % (i))
    print(f"Final Training Loss: {history.history['loss'][-1]}")
    print(f"Final Training Accuracy: {history.history['accuracy'][-1]}")
    print(f"Final Validation Loss: {history.history['val_loss'][-1]}")
    print(f"Final Validation Accuracy: {history.history['val_accuracy'][-1]}")
    visualize_history(history, f'{title_appendix}_adam={i}_finacc={history.history["val_accuracy"][-1]:.2f}')
    
    # Free Memory for following models
    del model
    gc.collect()
    tf.compat.v1.reset_default_graph()  # Reset the TensorFlow graph

57/57 [==============================] - 1s 11ms/step - loss: 2.9410 - accuracy: 0.0601
Adam starting learning rate: 0
Final Training Loss: 2.9410290718078613
Final Training Accuracy: 0.052586205303668976
Final Validation Loss: 2.941659450531006
Final Validation Accuracy: 0.05241379141807556
Test Accuracy: 6.01 %
Test Loss: 2.94
57/57 [==============================] - 1s 11ms/step - loss: 2.1533 - accuracy: 0.4708
Adam starting learning rate: 0
Final Training Loss: 0.7707065939903259
Final Training Accuracy: 0.7312068939208984
Final Validation Loss: 2.234323024749756
Final Validation Accuracy: 0.46482759714126587
Test Accuracy: 47.08 %
Test Loss: 2.15
57/57 [==============================] - 1s 11ms/step - loss: 1.3895 - accuracy: 0.7503
Adam starting learning rate: 0
Final Training Loss: 2.9437454941216856e-05
Final Training Accuracy: 1.0
Final Validation Loss: 1.378556251525879
Final Validation Accuracy: 0.7551724314689636
Test Accuracy: 75.03 %
Test Loss: 1.39
57/57 [==============

Die Plots für die Genauigkeit und den Loss legen nahe das für eine anfängliche Lernrate von 0.00005 noch mehr drin ist. Ich bin neugierig ...

In [9]:
# Adam Optimizer with variing starting learn rates
# convolutional filters: 32; epochs: 250; density: 500, rms_propagation learning rate:
title_appendix = 'l1_e250_d500'
clear_session() # clear keras session
tf.compat.v1.reset_default_graph()  # Reset the TensorFlow graph
#tf.config.gpu.set_per_process_memory_fraction(0.5)
# Build the model
model = Sequential()
model.add(Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(256,128,1)))
model.add(MaxPooling2D(2, 2))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dense(19, activation='softmax'))

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.00005)
model.compile(loss='categorical_crossentropy',
          optimizer=optimizer,
          metrics=['accuracy'])

# Fit the model
history = model.fit(X_train, y_train, epochs=250, batch_size=10, verbose=0, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print('Adam starting learning rate: 0.0005')
print(f"Final Training Loss: {history.history['loss'][-1]}")
print(f"Final Training Accuracy: {history.history['accuracy'][-1]}")
print(f"Final Validation Loss: {history.history['val_loss'][-1]}")
print(f"Final Validation Accuracy: {history.history['val_accuracy'][-1]}")
visualize_history(history, f'{title_appendix}_adam={i}_finacc={history.history["val_accuracy"][-1]:.2f}')

# Free Memory for following models
del model
gc.collect()
tf.compat.v1.reset_default_graph()  # Reset the TensorFlow graph

57/57 [==============================] - 1s 10ms/step - loss: 1.6676 - accuracy: 0.6251
Adam starting learning rate: 0.0005
Final Training Loss: 0.12450646609067917
Final Training Accuracy: 0.9863793253898621
Final Validation Loss: 1.5002870559692383
Final Validation Accuracy: 0.6324138045310974


Die neuen Graphen zeigen das Model sich bei 60% Genauigkeit einpendelt während der Validation-Loss sich nach ca. 200 Epochen wieder nach oben bewegt. 

In [ ]:
# Varrie the number of layers
# convolutional filters: 32; epochs: 100; density: 250, rms_propagation learning rate:
title_appendix = 'e100_d250'
for i in range(15):
    clear_session() # clear keras session
    tf.compat.v1.reset_default_graph()  # Reset the TensorFlow graph
    #tf.config.gpu.set_per_process_memory_fraction(0.5)
    # Build the model
    model = Sequential()
    model.add(Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(256,128,1)))
    model.add(MaxPooling2D(2, 2))
    for x in range(i):
        model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
        model.add(MaxPooling2D(2, 2))
    model.add(Flatten())
    model.add(Dense(250, activation='relu'))
    model.add(Dense(19, activation='softmax'))

    # Compile the model
    model.compile(loss='categorical_crossentropy',
                  optimizer=RMSprop(learning_rate=0.001),
                  metrics=['accuracy'])

    # Fit the model
    history = model.fit(X_train, y_train, epochs=100, batch_size=10, verbose=0, validation_split=0.2)

    # Evaluate the model
    loss, accuracy = model.evaluate(X_test, y_test)
    print('Number of layers: %.i' % (i))
    print(f"Final Training Loss: {history.history['loss'][-1]}")
    print(f"Final Training Accuracy: {history.history['accuracy'][-1]}")
    print(f"Final Validation Loss: {history.history['val_loss'][-1]}")
    print(f"Final Validation Accuracy: {history.history['val_accuracy'][-1]}")
    visualize_history(history, f'{title_appendix}_layercount={i}_finacc={history.history["val_accuracy"][-1]:.2f}')
    
    # Free Memory for following models
    del model
    gc.collect()
    tf.compat.v1.reset_default_graph()  # Reset the TensorFlow graph

In [ ]:
# Final model results:
print('Test Accuracy: %.2f %%' % (accuracy*100))
print('Test Loss: %.2f' % (loss))

In [14]:
# Clear grafic memory
clear_session() # clear keras session
tf.compat.v1.reset_default_graph()  # Reset the TensorFlow graph
# Free Memory for following models
del model
gc.collect()
tf.compat.v1.reset_default_graph()  # Reset the TensorFlow graph